In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
tf.keras.mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080 Ti, compute capability 8.6


In [2]:
# data loading config
batch_size = 128
img_height = 150
img_width = 150
dataPath = 'train'
labels = 'inferred'  # auto find from folders
categorical = 'categorical'  # one hot encoding
color_mode = 'rgb'
shuffle = True
seed = 69
test_split = 0.2  # split into train and test (NOT val), 0-1
AUTOTUNE = tf.data.AUTOTUNE

print('Training data')
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'train',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=batch_size,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode=categorical)

# Flow validation images in batches of 20 using test_datagen generator
print('Validation data')
validation_generator = test_datagen.flow_from_directory(
        'val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=categorical)

# train = tf.keras.preprocessing.image_dataset_from_directory(dataPath, labels=labels, label_mode=label_mode,
# color_mode=color_mode, shuffle=shuffle, subset="training", seed=seed, validation_split=test_split,
# image_size=(img_height, img_width), batch_size=batch_size)
# train_class_names = train.class_names
#
# val = tf.keras.preprocessing.image_dataset_from_directory(dataPath, labels=labels, label_mode=label_mode,
# color_mode=color_mode, shuffle=shuffle, subset="validation", seed=seed, validation_split=test_split,
# image_size=(img_height, img_width), batch_size=batch_size)


Training data
Found 20000 images belonging to 2 classes.
2
Validation data
Found 5000 images belonging to 2 classes.


In [3]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
epoch = 20
batch_size = 128
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('./best_model',monitor='val_accuracy',save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.05, patience=5, verbose=1)
]

# model layers
xInput = layers.Input((img_height, img_width, 3))  # get resolutions, ignore batch size
x = layers.Conv2D(16, 3, activation='relu')(xInput)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.Activation('relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Activation('relu')(x)
x = layers.Dense(128)(x)
x = layers.Activation('relu')(x)
xOutput = layers.Dense(train_generator.num_classes)(x)  # a probability for each class so need same no as classes

model = tf.keras.Model(xInput, xOutput)
model.compile(optimizer=opt, loss=loss, metrics='accuracy')
model.summary()
model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=train_generator.samples//batch_size, epochs=epoch, callbacks=callbacks, verbose=1)

C:\Program Files\Python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496 